# Importing libraries

In [109]:
from pyzotero import zotero
import os
import tweepy as tw
import pandas as pd
import datetime
import json, sys
from datetime import date, timedelta  
import datetime
import plotly.express as px
import pycountry
import re
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

In [110]:
"# All items in the Zotero Intelligence bibliography library".index("All")

2

In [111]:
library_id = '2514686'
library_type = 'group'
api_key = '' # api_key is only needed for private groups and libraries
zot = zotero.Zotero(library_id, library_type)


# All items in the Zotero Intelligence bibliography library

In [112]:
items = zot.everything(zot.top())

data3=[]
columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key', 'DOI']

for item in items:
    # Extracting various types of creators
    authors = ""
    if 'creators' in item['data']:
        authors_info = item['data']['creators']
        author_names = []
        for author in authors_info:
            creator_type = author.get('creatorType')
            valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster', 'presenter', 'translator', 'programmer']
            
            if item['data']['itemType'] == 'bookSection' and creator_type == 'editor':
                continue  # Skip adding 'editor' for Book chapters
            
            if creator_type in valid_types:
                if 'firstName' in author and 'lastName' in author:
                    author_names.append(author['firstName'] + ' ' + author['lastName'])
                elif 'name' in author:
                    author_names.append(author['name'])
                
        authors = ', '.join(author_names)
        
    data3.append((
        item['data']['title'], 
        item['data']['itemType'], 
        item['data']['url'], 
        item['data']['abstractNote'], 
        item['links']['alternate']['href'],
        item['data'].get('date'),
        authors,  # Add the concatenated authors string
        item['data'].get('publisher'),
        item['data'].get('publicationTitle'),
        item['data']['dateAdded'],
        item['data']['collections'],
        item['data'].get('DOI')
        ))
# pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data3, columns=columns3)

mapping_types = {
    'thesis': 'Thesis',
    'journalArticle': 'Journal article',
    'book': 'Book',
    'bookSection': 'Book chapter',
    'blogPost': 'Blog post',
    'videoRecording': 'Video',
    'podcast': 'Podcast',
    'magazineArticle': 'Magazine article',
    'webpage': 'Webpage',
    'newspaperArticle': 'Newspaper article',
    'report': 'Report',
    'forumPost': 'Forum post',
    'manuscript': 'Manuscript',
    'document': 'Document',
    'conferencePaper': 'Conference paper',
    'film': 'Film',
    'presentation': 'Presentation',
    'audioRecording':'Podcast',
    'preprint':'Preprint',
    'hearing':'Hearing',
    'computerProgram':'Computer program'
}
df['Publication type'] = df['Publication type'].replace(mapping_types)

mapping_publisher = {
    'Taylor & Francis Group': 'Taylor and Francis',
    'Taylor and Francis': 'Taylor and Francis',
    'Taylor & Francis': 'Taylor and Francis',
    'Routledge': 'Routledge',
    'Routledge Handbooks Online': 'Routledge',
    'Praeger Security International': 'Praeger',
    'Praeger': 'Praeger'
}
df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

mapping_journal = {
    'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and Counter Intelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'Intelligence and national security': 'Intelligence and National Security',
    'Intelligence and National Security': 'Intelligence and National Security',
    'Intelligence & National Security': 'Intelligence and National Security'
}

df['Journal'] = df['Journal'].replace(mapping_journal)

# df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [113]:
df_test = df[df['Title']=='Intelligence Practice and Democratic Oversight']
df_test

,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI
481,Intelligence Practice and Democratic Oversight,Report,https://www.dcaf.ch/intelligence-practice-and-...,,https://www.zotero.org/groups/intelligence_bib...,2003-07-01,Geneva Centre for Security Sector Governance,None,None,2024-02-19T11:03:09Z,[DVEM4H4W],None


In [114]:
# items = zot.everything(zot.top())

# data3=[]
# columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key']

# for item in items:
#     # Extracting various types of creators
#     authors = ""
#     if 'creators' in item['data']:
#         authors_info = item['data']['creators']
#         author_names = []
#         for author in authors_info:
#             creator_type = author.get('creatorType')
#             valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster']
#             if creator_type in valid_types:  # Include all identified creator types
#                 if 'firstName' in author and 'lastName' in author:
#                     author_names.append(author['firstName'] + ' ' + author['lastName'])
#                 elif 'name' in author:
#                     author_names.append(author['name'])  # Handling cases where 'firstName' and 'lastName' are absent
                
#         authors = ', '.join(author_names)
        
#     data3.append((
#         item['data']['title'], 
#         item['data']['itemType'], 
#         item['data']['url'], 
#         item['data']['abstractNote'], 
#         item['links']['alternate']['href'],
#         item['data'].get('date'),
#         authors,  # Add the concatenated authors string
#         item['data'].get('publisher'),
#         item['data'].get('publicationTitle'),
#         item['data']['dateAdded'],
#         item['data']['collections']
#         )) 
# # pd.set_option('display.max_colwidth', None)

# df = pd.DataFrame(data3, columns=columns3)

# mapping_types = {
#     'thesis': 'Thesis',
#     'journalArticle': 'Journal article',
#     'book': 'Book',
#     'bookSection': 'Book chapter',
#     'blogPost': 'Blog post',
#     'videoRecording': 'Video',
#     'podcast': 'Podcast',
#     'magazineArticle': 'Magazine article',
#     'webpage': 'Webpage',
#     'newspaperArticle': 'Newspaper article',
#     'report': 'Report',
#     'forumPost': 'Forum post',
#     'manuscript': 'Manuscript',
#     'document': 'Document',
#     'conferencePaper': 'Conference paper',
#     'film': 'Film',
#     'presentation': 'Presentation',
#     'audioRecording':'Podcast'
# }
# df['Publication type'] = df['Publication type'].replace(mapping_types)

# mapping_publisher = {
#     'Taylor & Francis Group': 'Taylor and Francis',
#     'Taylor and Francis': 'Taylor and Francis',
#     'Taylor & Francis': 'Taylor and Francis',
#     'Routledge': 'Routledge',
#     'Routledge Handbooks Online': 'Routledge',
#     'Praeger Security International': 'Praeger',
#     'Praeger': 'Praeger'
# }
# df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# # df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# # df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# # df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

# mapping_journal = {
#     'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'Intelligence and national security': 'Intelligence and National Security',
#     'Intelligence and National Security': 'Intelligence and National Security',
#     'Intelligence & National Security': 'Intelligence and National Security'
# }

# df['Journal'] = df['Journal'].replace(mapping_journal)

# # df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# # df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [115]:
def zotero_collections2(library_id, library_type):
    collections = zot.collections()
    data = [(item['data']['key'], item['data']['name'], item['meta']['numItems'], item['links']['alternate']['href']) for item in collections]
    df_collections = pd.DataFrame(data, columns=['Key', 'Name', 'Number', 'Link'])
    return df_collections
df_collections_2 = zotero_collections2(library_id, library_type)

In [116]:
def duplicate_rows_by_col_key(df, df_collections):
    # Duplicate rows based on 'Col key'
    duplicated_rows = []
    for index, row in df.iterrows():
        if isinstance(row['Col key'], list):
            for key in row['Col key']:
                new_row = row.copy()
                collection_info = df_collections[df_collections['Key'] == key]
                if not collection_info.empty:
                    new_row['Collection_Key'] = key
                    new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                    new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                    duplicated_rows.append(new_row)
        else:
            key = row['Col key']
            new_row = row.copy()
            collection_info = df_collections[df_collections['Key'] == key]
            if not collection_info.empty:
                new_row['Collection_Key'] = key
                new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                duplicated_rows.append(new_row)

    # Create a new DataFrame with duplicated rows
    duplicated_df = pd.DataFrame(duplicated_rows)

    return duplicated_df

# Duplicating rows based on 'Col key' and collections information
duplicated_df = duplicate_rows_by_col_key(df, df_collections_2)

In [117]:
# day_allitems = datetime.date.today().isoformat()

In [118]:
# df_fa = df['FirstName2']
# df_fa = pd.DataFrame(df_fa.tolist())
# df_fa = df_fa[0]
# df_fa = df_fa.apply(lambda x: {} if pd.isna(x) else x)
# df_new = pd.json_normalize(df_fa, errors='ignore') 
# df = pd.concat([df, df_new], axis=1)
# df['firstName'] = df['firstName'].fillna('null')
# df['lastName'] = df['lastName'].fillna('null')

In [119]:
df.to_csv('all_items.csv')
duplicated_df.to_csv('all_items_duplicated.csv')

In [120]:
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras

# Countries 

In [121]:
# Read the csv file into a pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')

# Dictionary to map non-proper country names to their proper names
country_map = {
    'british': 'UK',
    'great britain': 'UK',
    'UK' : 'UK', 
    'america' : 'United States',
    'United States of America' : 'United States',
    'Soviet Union': 'Russia', 
    'american' : 'United States',
    'United States' : 'United States',
    'russian' : 'Russia'
    # Add more mappings as needed
}

# Find the country names in the "title" column of the dataframe
found_countries = {}
for i, row in df.iterrows():
    title = str(row['Title']).lower()
    for country in pycountry.countries:
        name = country.name.lower()
        if name in title or (name + 's') in title:  # Check for singular and plural forms of country names
            proper_name = country.name
            found_countries[proper_name] = found_countries.get(proper_name, 0) + 1
    for non_proper, proper in country_map.items():
        if non_proper in title:
            found_countries[proper] = found_countries.get(proper, 0) + title.count(non_proper)

# Create a new dataframe containing the found countries and their counts
df_countries = pd.DataFrame({'Country': list(found_countries.keys()), 'Count': list(found_countries.values())})

In [122]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [123]:
df_countries=df_countries.sort_values(by='Count', ascending=False)
df_countries.reset_index(drop=True)

,Country,Count
0,United States,647
1,UK,455
2,Israel,143
3,Russia,131
4,Japan,111
...,...,...
104,Ethiopia,1
105,Kazakhstan,1
106,Nicaragua,1
107,Namibia,1


In [124]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [125]:
df_countries.to_csv('countries.csv',index=False)

# NER analysis

In [126]:
import spacy
import nltk
import ast
from spacy.pipeline import EntityRecognizer

In [127]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "MI6"}]
ruler.add_patterns(patterns)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
c:\Users\yaozk\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:865: UserWarning:

[W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible

In [128]:
def extract_entities(text):
    doc = nlp(text)
    orgs = []
    gpes = []
    people = []
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            orgs.append(entity.text)
        elif entity.label_ == 'GPE':
            gpes.append(entity.text)
        elif entity.label_ == 'PERSON':
            people.append(entity.text)
    return pd.Series({'ORG': orgs, 'GPE': gpes, 'PERSON': people})

In [129]:
df_title = df[['Title']].copy()
df_title = df_title.rename(columns={'Title':'Text'})
df_abstract = df[['Abstract']].copy()
df_abstract = df_abstract.rename(columns={'Abstract':'Text'})
df_one = pd.concat([df_title, df_abstract], ignore_index=True)
df_one['Text'] = df_one['Text'].fillna('')

In [130]:
df_one = pd.concat([df_one[['Text']], df_one['Text'].apply(extract_entities)], axis=1)
df_one = df_one.explode('GPE').reset_index(drop=True)
df_one = df_one.explode('ORG').reset_index(drop=True)
df_one = df_one.explode('PERSON').reset_index(drop=True)

In [131]:
df_one_g = df_one.copy()
df_one_g = df_one[['Text', 'GPE']]
# df_one_g = df_one_g.fillna('')
df_one_g = df_one_g.drop_duplicates(subset=['Text', 'GPE'])

gpe_counts = df_one_g['GPE'].value_counts().reset_index()
gpe_counts.columns = ['GPE', 'count']

# pd.options.display.max_rows = None


mapping_locations = {
    'the United States': 'USA',
    'The United States': 'USA',
    'US': 'USA',
    'U.S.': 'USA',
    'United States' : 'USA',
    'America' : 'USA',
    'the United States of America' : 'USA',
    'Britain' : 'UK',
    'the United Kingdom': 'UK',
    'U.K.' : 'UK',
    'Global Britain' : 'UK',
    'United Kingdom' : 'UK', 
    'the Soviet Union' : 'Russia',
    'The Soviet Union' : 'Russia',
    'USSR' : 'Russia',
    'Ukraine - Perspective' : 'Ukraine',
    'Ukrainian' : 'Ukraine',
    'Great Britain' : 'UK',
    'Ottoman Empire' : 'Turkey'
}
gpe_counts['GPE'] =gpe_counts['GPE'].replace(mapping_locations)
gpe_counts = gpe_counts.groupby('GPE').sum().reset_index()
gpe_counts.sort_values('count', ascending=False, inplace=True)
gpe_counts = gpe_counts.reset_index(drop=True)
gpe_counts.head(15)

,GPE,count
0,USA,1334
1,UK,687
2,Russia,485
3,Germany,203
4,Ukraine,193
5,Israel,177
6,China,155
7,France,142
8,Canada,137
9,Iraq,131


In [132]:
df_one_p = df_one.copy()
df_one_p = df_one[['Text', 'PERSON']]
# df_one_p = df_one_g.fillna('')
df_one_p = df_one_p.drop_duplicates(subset=['Text', 'PERSON'])

person_counts = df_one_p['PERSON'].value_counts().reset_index()
person_counts.columns = ['PERSON', 'count']

mapping_person = {
    'Putin' : 'Vladimir Putin',
    'Vladimir Putin' : 'Vladimir Putin',
    'Churchill' : 'Winston Churchill',
    'Hitler' : 'Adolf Hitler',
    'Biden' : 'Joe Biden',
    "John le Carré’s" : "John le Carré"
}

person_counts['PERSON'] =person_counts['PERSON'].replace(mapping_person)
person_counts = person_counts.groupby('PERSON').sum().reset_index()
person_counts.sort_values('count', ascending=False, inplace=True)

remove_person = ['MI6', 'Twitter', 'GRU'
          ]
person_counts = person_counts[~person_counts['PERSON'].isin(remove_person)]
person_counts = person_counts.reset_index(drop=True)

person_counts.head(15)

,PERSON,count
0,Adolf Hitler,82
1,Vladimir Putin,70
2,Winston Churchill,51
3,Stalin,39
4,Eisenhower,29
5,Andrew Hammond,28
6,Mujibur Rahman,28
7,Bush,27
8,John le Carré,24
9,James Bond,24


In [133]:
df_one_o = df_one.copy()
df_one_o = df_one[['Text', 'ORG']]
# df_one_p = df_one_g.fillna('')
df_one_o = df_one_o.drop_duplicates(subset=['Text', 'ORG'])

org_counts = df_one_o['ORG'].value_counts().reset_index()
org_counts.columns = ['ORG', 'count']

mapping_organisations = {
    'The British Secret Intelligence Service' : 'SIS',
    'the British Secret Intelligence Service' : 'SIS',
    'The Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee - History' : 'Joint Intelligence Committee',
    'Central Intelligence Agency' : 'CIA',
    'the Central Intelligence Agency' : 'CIA',
    'the Foreign Office' : 'Foreign Office',
    'Schar School' : 'Schar School of Policy and Government',
    'the Secret Intelligence Service' : 'SIS',
    "George Mason University's" : "George Mason University",
    'JIC' : 'Joint Intelligence Committee'
}
org_counts['ORG'] =org_counts['ORG'].replace(mapping_organisations)
org_counts = org_counts.groupby('ORG').sum().reset_index()
org_counts.sort_values('count', ascending=False, inplace=True)

remove_orgs = ['Intelligence', 'Kremlin', 'Ultra', 'International Security', 'Intelligence Studies', 'Intelligence Analysis', 'OSINT']
org_counts = org_counts[~org_counts['ORG'].isin(remove_orgs)]
org_counts = org_counts.reset_index(drop=True)

org_counts.head(15)

,ORG,count
0,CIA,670
1,KGB,133
2,FBI,97
3,Allied,94
4,Congress,70
5,SIS,65
6,SOE,63
7,the Intelligence Community,61
8,NATO,57
9,OSS,53


In [134]:
gpe_counts.head(15).to_csv('gpe.csv')
person_counts.head(15).to_csv('person.csv')
org_counts.head(15).to_csv('org.csv')

In [135]:
test = pd.read_csv('all_items.csv')
test

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI
0,0,“NATO’s Assistant Secretary General for Intell...,Podcast,https://thecyberwire.com/podcasts/spycast/524/...,David Cattler joins Andrew Hammond to discuss ...,https://www.zotero.org/groups/intelligence_bib...,2022-01-02,David Cattler,NaN,NaN,2024-03-03T23:16:06Z,['CK5MNYPQ'],NaN
1,1,“Keeping Secrets/Disclosing Secrets” – with Sp...,Podcast,https://thecyberwire.com/podcasts/spycast/526/...,David Fricker had the No.2 job at Australia’s ...,https://www.zotero.org/groups/intelligence_bib...,2022-02-15,David Fricker,NaN,NaN,2024-03-03T23:14:42Z,['KGU8VLSW'],NaN
2,2,“The National Intelligence University” – with ...,Podcast,https://thecyberwire.com/podcasts/spycast/527/...,J. Scott Cameron joins Andrew to discuss the N...,https://www.zotero.org/groups/intelligence_bib...,2022-02-22,Scott Cameron,NaN,NaN,2024-03-03T23:14:03Z,['H28QZ8XV'],NaN
3,3,"""From the CIA to Strategic Cyber"" - with Hans ...",Podcast,https://thecyberwire.com/podcasts/spycast/529/...,Hans Holmer joins Andrew to discuss his time a...,https://www.zotero.org/groups/intelligence_bib...,2022-03-08,Hans Holmer,NaN,NaN,2024-03-03T23:13:03Z,['8XXD789V'],NaN
4,4,"""So, I Design Board Games for the CIA..."" - wi...",Podcast,https://thecyberwire.com/podcasts/spycast/530/...,Volko Ruhnke joins Andrew to discuss his life ...,https://www.zotero.org/groups/intelligence_bib...,2022-03-15,Volko Ruhnke,NaN,NaN,2024-03-03T23:11:12Z,['CZJ36V8L'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,6424,"Analyzing Intelligence: Origins, Obstacles, an...",Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2008,"Roger Z. George, James B. Bruce",Georgetown University Press,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN
6425,6425,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN
6426,6426,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN
6427,6427,Silent Warfare: Understanding the World of Int...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN


# Retrieving citation count and open access status

In [136]:
df_doi = pd.read_csv('all_items.csv')
if datetime.today().weekday() == 4:

    df_doi = df_doi[['Zotero link', 'DOI']].dropna()
    df_doi = df_doi.drop(df_doi[df_doi['DOI'] == ''].index)
    df_doi = df_doi.reset_index(drop=True)
    df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')

    def fetch_article_metadata(doi):
        base_url = 'https://api.openalex.org/works/https://doi.org/'
        response = requests.get(base_url + doi)
        if response.status_code == 200:
            data = response.json()
            counts_by_year = data.get('counts_by_year', [])
            if counts_by_year:
                first_citation_year = min(entry.get('year') for entry in data['counts_by_year'])
            else:
                first_citation_year = None
            if data.get('counts_by_year'):
                last_citation_year = max(entry.get('year') for entry in data['counts_by_year'])
            else:
                last_citation_year = None
    
            article_metadata = {
                'ID': data.get('id'),
                'Citation': data.get('cited_by_count'),
                'OA status': data.get('open_access', {}).get('is_oa'),
                'Citation_list': data.get('cited_by_api_url'),
                'First_citation_year': first_citation_year,
                'Last_citation_year':last_citation_year,
                'Publication_year':data.get('publication_year'),
                'OA_link':data.get('open_access', {}).get('oa_url')
            }
            return article_metadata
        else:
            return {'ID': None, 'Citation': None, 'OA status': None, 'First_citation_year': None, 'Last_citation_year':None, 'Publication_year' : None, 'OA_link':None}
    df_doi['ID'] = None
    df_doi['Citation'] = None
    df_doi['OA status'] = None
    df_doi['Citation_list'] = None
    df_doi['First_citation_year'] = None
    df_doi['Last_citation_year']=None
    df_doi['Publication_year'] = None
    df_doi['OA_link'] = None

    # Iterate over each row in the DataFrame
    for index, row in df_doi.iterrows():
        doi = row['DOI']
        article_metadata = fetch_article_metadata(doi)
        if article_metadata:
            # Update DataFrame with fetched information
            df_doi.at[index, 'ID'] = article_metadata['ID']
            df_doi.at[index, 'Citation'] = article_metadata['Citation']
            df_doi.at[index, 'OA status'] = article_metadata['OA status']
            df_doi.at[index, 'First_citation_year'] = article_metadata['First_citation_year']
            df_doi.at[index, 'Last_citation_year'] = article_metadata['Last_citation_year']
            if 'Citation_list' in article_metadata:
                df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
            else:
                # Set to None if key is not found
                df_doi.at[index, 'Citation_list'] = None
            df_doi.at[index, 'Publication_year'] = article_metadata['Publication_year']
            if 'OA_link' in article_metadata:
                df_doi.at[index, 'OA_link'] = article_metadata['OA_link']
            else:
                # Set to None if key is not found
                df_doi.at[index, 'OA_link'] = None

    # Calculate the difference between First_citation_year and Publication_year
    df_doi['Year_difference'] = df_doi['First_citation_year'] - df_doi['Publication_year']
    df_doi.to_csv('citations.csv')
else:
    print('This part will be refreshed every Wednesday!')
df_doi

This part will be refreshed every Wednesday!


,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI
0,0,“NATO’s Assistant Secretary General for Intell...,Podcast,https://thecyberwire.com/podcasts/spycast/524/...,David Cattler joins Andrew Hammond to discuss ...,https://www.zotero.org/groups/intelligence_bib...,2022-01-02,David Cattler,NaN,NaN,2024-03-03T23:16:06Z,['CK5MNYPQ'],NaN
1,1,“Keeping Secrets/Disclosing Secrets” – with Sp...,Podcast,https://thecyberwire.com/podcasts/spycast/526/...,David Fricker had the No.2 job at Australia’s ...,https://www.zotero.org/groups/intelligence_bib...,2022-02-15,David Fricker,NaN,NaN,2024-03-03T23:14:42Z,['KGU8VLSW'],NaN
2,2,“The National Intelligence University” – with ...,Podcast,https://thecyberwire.com/podcasts/spycast/527/...,J. Scott Cameron joins Andrew to discuss the N...,https://www.zotero.org/groups/intelligence_bib...,2022-02-22,Scott Cameron,NaN,NaN,2024-03-03T23:14:03Z,['H28QZ8XV'],NaN
3,3,"""From the CIA to Strategic Cyber"" - with Hans ...",Podcast,https://thecyberwire.com/podcasts/spycast/529/...,Hans Holmer joins Andrew to discuss his time a...,https://www.zotero.org/groups/intelligence_bib...,2022-03-08,Hans Holmer,NaN,NaN,2024-03-03T23:13:03Z,['8XXD789V'],NaN
4,4,"""So, I Design Board Games for the CIA..."" - wi...",Podcast,https://thecyberwire.com/podcasts/spycast/530/...,Volko Ruhnke joins Andrew to discuss his life ...,https://www.zotero.org/groups/intelligence_bib...,2022-03-15,Volko Ruhnke,NaN,NaN,2024-03-03T23:11:12Z,['CZJ36V8L'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,6424,"Analyzing Intelligence: Origins, Obstacles, an...",Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2008,"Roger Z. George, James B. Bruce",Georgetown University Press,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN
6425,6425,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN
6426,6426,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN
6427,6427,Silent Warfare: Understanding the World of Int...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN


In [137]:
# ## Remove after the first successful retrieval 2024-02-18

# df_doi = pd.read_csv('all_items.csv')
# if datetime.today().weekday() == 3:

#     df_doi = df_doi[['Zotero link', 'DOI']].dropna()
#     df_doi = df_doi.drop(df_doi[df_doi['DOI'] == ''].index)
#     df_doi = df_doi.reset_index(drop=True)
#     df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')

#     def fetch_article_metadata(doi):
#         base_url = 'https://api.openalex.org/works/https://doi.org/'
#         response = requests.get(base_url + doi)
#         if response.status_code == 200:
#             data = response.json()
#             counts_by_year = data.get('counts_by_year', [])
#             if counts_by_year:
#                 first_citation_year = min(entry.get('year') for entry in data['counts_by_year'])
#             else:
#                 first_citation_year = None
#             article_metadata = {
#                 'ID': data.get('id'),
#                 'Citation': data.get('cited_by_count'),
#                 'OA status': data.get('open_access', {}).get('is_oa'),
#                 'Citation_list': data.get('cited_by_api_url'),
#                 'First_citation_year': first_citation_year,
#                 'Publication_year':data.get('publication_year')
#             }
#             return article_metadata
#         else:
#             return {'ID': None, 'Citation': None, 'OA status': None, 'First_citation_year': None, 'Publication_year' : None}
#     df_doi['ID'] = None
#     df_doi['Citation'] = None
#     df_doi['OA status'] = None
#     df_doi['Citation_list'] = None
#     df_doi['First_citation_year'] = None
#     df_doi['Publication_year'] = None

#     # Iterate over each row in the DataFrame
#     for index, row in df_doi.iterrows():
#         doi = row['DOI']
#         article_metadata = fetch_article_metadata(doi)
#         if article_metadata:
#             # Update DataFrame with fetched information
#             df_doi.at[index, 'ID'] = article_metadata['ID']
#             df_doi.at[index, 'Citation'] = article_metadata['Citation']
#             df_doi.at[index, 'OA status'] = article_metadata['OA status']
#             df_doi.at[index, 'First_citation_year'] = article_metadata['First_citation_year']
#             if 'Citation_list' in article_metadata:
#                 df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
#             else:
#                 # Set to None if key is not found
#                 df_doi.at[index, 'Citation_list'] = None
#             df_doi.at[index, 'Publication_year'] = article_metadata['Publication_year']

#     # Calculate the difference between First_citation_year and Publication_year
#     df_doi['Year_difference'] = df_doi['First_citation_year'] - df_doi['Publication_year']
#     df_doi.to_csv('citations.csv')
# else:
#     print('This part will be refreshed every Wednesday!')
# df_doi

In [138]:
# ## Retrieving citation and open access status without first ciation year
# df_doi = pd.read_csv('all_items.csv')
# # df_doi = df_doi.head(10)
# if datetime.today().weekday() == 5:

#     df_doi = df_doi[['Zotero link', 'DOI']].dropna()
#     df_doi = df_doi.drop(df_doi[df_doi['DOI']==''].index)
#     df_doi = df_doi.reset_index(drop=True)
#     df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')
#     df_doi

#     def fetch_article_metadata(doi):
#         base_url = 'https://api.openalex.org/works/https://doi.org/'
#         response = requests.get(base_url + doi)
#         if response.status_code == 200:
#             data = response.json()
#             article_metadata = {
#                 'ID': data.get('id'),
#                 'Citation': data.get('cited_by_count'),
#                 'OA status': data.get('open_access', {}).get('is_oa'),
#                 'Citation_list':data.get('cited_by_api_url')
#             }
#             return article_metadata
#         else:
#             return {'ID': None, 'Citation': None, 'OA status': None}

#     df_doi['ID'] = None
#     df_doi['Citation'] = None
#     df_doi['OA status'] = None
#     df_doi['Citation_list'] = None

#     # Iterate over each row in the DataFrame
#     for index, row in df_doi.iterrows():
#         doi = row['DOI']
#         article_metadata = fetch_article_metadata(doi)
#         if article_metadata:
#             # Update DataFrame with fetched information
#             df_doi.at[index, 'ID'] = article_metadata['ID']
#             df_doi.at[index, 'Citation'] = article_metadata['Citation']
#             df_doi.at[index, 'OA status'] = article_metadata['OA status']
#             if 'Citation_list' in article_metadata:
#                 df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
#             else:
#                 # Set to None if key is not found
#                 df_doi.at[index, 'Citation_list'] = None
#     df_doi
#     df_doi.to_csv('citations.csv')
# else:
#     print('This part will be refreshed every Wednesday!')

In [139]:
df_doi

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI
0,0,“NATO’s Assistant Secretary General for Intell...,Podcast,https://thecyberwire.com/podcasts/spycast/524/...,David Cattler joins Andrew Hammond to discuss ...,https://www.zotero.org/groups/intelligence_bib...,2022-01-02,David Cattler,NaN,NaN,2024-03-03T23:16:06Z,['CK5MNYPQ'],NaN
1,1,“Keeping Secrets/Disclosing Secrets” – with Sp...,Podcast,https://thecyberwire.com/podcasts/spycast/526/...,David Fricker had the No.2 job at Australia’s ...,https://www.zotero.org/groups/intelligence_bib...,2022-02-15,David Fricker,NaN,NaN,2024-03-03T23:14:42Z,['KGU8VLSW'],NaN
2,2,“The National Intelligence University” – with ...,Podcast,https://thecyberwire.com/podcasts/spycast/527/...,J. Scott Cameron joins Andrew to discuss the N...,https://www.zotero.org/groups/intelligence_bib...,2022-02-22,Scott Cameron,NaN,NaN,2024-03-03T23:14:03Z,['H28QZ8XV'],NaN
3,3,"""From the CIA to Strategic Cyber"" - with Hans ...",Podcast,https://thecyberwire.com/podcasts/spycast/529/...,Hans Holmer joins Andrew to discuss his time a...,https://www.zotero.org/groups/intelligence_bib...,2022-03-08,Hans Holmer,NaN,NaN,2024-03-03T23:13:03Z,['8XXD789V'],NaN
4,4,"""So, I Design Board Games for the CIA..."" - wi...",Podcast,https://thecyberwire.com/podcasts/spycast/530/...,Volko Ruhnke joins Andrew to discuss his life ...,https://www.zotero.org/groups/intelligence_bib...,2022-03-15,Volko Ruhnke,NaN,NaN,2024-03-03T23:11:12Z,['CZJ36V8L'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424,6424,"Analyzing Intelligence: Origins, Obstacles, an...",Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2008,"Roger Z. George, James B. Bruce",Georgetown University Press,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN
6425,6425,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN
6426,6426,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN
6427,6427,Silent Warfare: Understanding the World of Int...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN


In [140]:
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2

,Zotero link,DOI,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2024.2302294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.zotero.org/groups/intelligence_bib...,10.1080/02684527.2024.2321692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.zotero.org/groups/intelligence_bib...,10.1080/02684527.2024.2321693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.zotero.org/groups/intelligence_bib...,10.34007/jehss.v6i3.2088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.zotero.org/groups/intelligence_bib...,10.1080/0161-117791833174,https://openalex.org/W2031997701,1.0,False,https://api.openalex.org/works?filter=cites:W2...,2015.0,2015.0,1977.0,NaN,38.0
...,...,...,...,...,...,...,...,...,...,...,...
3475,https://www.zotero.org/groups/intelligence_bib...,10.1177/1755088217712069,https://openalex.org/W2612406272,4.0,True,https://api.openalex.org/works?filter=cites:W2...,2019.0,2022.0,2017.0,https://journals.sagepub.com/doi/pdf/10.1177/1...,2.0
3476,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2019.1660951,https://openalex.org/W2996219269,5.0,False,https://api.openalex.org/works?filter=cites:W2...,2021.0,2023.0,2019.0,NaN,2.0
3477,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2012.705186,https://openalex.org/W2033999438,26.0,False,https://api.openalex.org/works?filter=cites:W2...,2013.0,2022.0,2013.0,NaN,0.0
3478,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2020.1748993,https://openalex.org/W3022421216,0.0,False,https://api.openalex.org/works?filter=cites:W3...,NaN,NaN,2020.0,NaN,NaN


In [141]:
df_1 = pd.read_csv('all_items.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = df_doi_1.drop(columns=['DOI'])
df_1 = pd.merge(df_1, df_doi_1, on='Zotero link', how='left')
df_1.to_csv('all_items.csv')

In [142]:
df_2 = pd.read_csv('all_items_duplicated.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = df_doi_2.drop(columns=['DOI'])
df_2 = pd.merge(df_2, df_doi_2, on='Zotero link', how='left')
df_2.to_csv('all_items_duplicated.csv')

In [143]:
df_2 = pd.read_csv('all_items_duplicated.csv')
df_2

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,...,Collection_Link,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,0,“NATO’s Assistant Secretary General for Intell...,Podcast,https://thecyberwire.com/podcasts/spycast/524/...,David Cattler joins Andrew Hammond to discuss ...,https://www.zotero.org/groups/intelligence_bib...,2022-01-02,David Cattler,NaN,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,“Keeping Secrets/Disclosing Secrets” – with Sp...,Podcast,https://thecyberwire.com/podcasts/spycast/526/...,David Fricker had the No.2 job at Australia’s ...,https://www.zotero.org/groups/intelligence_bib...,2022-02-15,David Fricker,NaN,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,“The National Intelligence University” – with ...,Podcast,https://thecyberwire.com/podcasts/spycast/527/...,J. Scott Cameron joins Andrew to discuss the N...,https://www.zotero.org/groups/intelligence_bib...,2022-02-22,Scott Cameron,NaN,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"""From the CIA to Strategic Cyber"" - with Hans ...",Podcast,https://thecyberwire.com/podcasts/spycast/529/...,Hans Holmer joins Andrew to discuss his time a...,https://www.zotero.org/groups/intelligence_bib...,2022-03-08,Hans Holmer,NaN,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"""So, I Design Board Games for the CIA..."" - wi...",Podcast,https://thecyberwire.com/podcasts/spycast/530/...,Volko Ruhnke joins Andrew to discuss his life ...,https://www.zotero.org/groups/intelligence_bib...,2022-03-15,Volko Ruhnke,NaN,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9621,9621,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9622,9622,Silent Warfare: Understanding the World of Int...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9623,9623,Silent Warfare: Understanding the World of Int...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9624,9624,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
ttt = df_2[df_2['DOI']=='10.1080/08850600600829882']
ttt

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,...,Collection_Link,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
6911,6911,Modeling an Intelligence Analysis Profession o...,Journal article,https://doi.org/10.1080/08850600600829882,NaN,https://www.zotero.org/groups/intelligence_bib...,2006-12-01,"Stephen Marrin, Jonathan D. Clemente",NaN,Intl Journal of Intelligence and Counterintell...,...,https://www.zotero.org/groups/intelligence_bib...,https://openalex.org/W2074061346,16.0,True,https://api.openalex.org/works?filter=cites:W2...,2012.0,2023.0,2006.0,http://www.stuart-hall.com/ftp/IJIC--Vol19No4-...,6.0
